In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
from ling_features import word_count, number_count, sent_count

conn_string = 'postgresql://10.101.13.99/crsp'

# Doesn't need to expand_json for this one
"""def expand_json(df, col):
    return pd.concat([df.drop([col], axis=1),
                      df[col].map(lambda x: json.loads(json.dumps(x))).apply(pd.Series)], axis=1)"""

engine = create_engine(conn_string)
        
sql =  """
SELECT file_name, last_update, speaker_number, context, section,
   count(speaker_text) AS count,
   string_agg(speaker_text, ' ')  AS speaker_text
FROM streetevents.speaker_data
WHERE file_name = '1000000_T' 
GROUP BY file_name, last_update, speaker_name, speaker_number, context, section
ORDER BY section, context, speaker_number"""

speaker_data = pd.read_sql(sql, engine)

In [2]:
speaker_data['last_update'] = speaker_data['last_update'].map(lambda x: str(x.astimezone('UTC')))
speaker_data['sum'] = speaker_data['speaker_text'].apply(word_count)
speaker_data['sent_count'] = speaker_data['speaker_text'].apply(sent_count)
speaker_data['sum_6'] = speaker_data['speaker_text'].apply(word_count, args = (6,))
speaker_data['sum_num'] = speaker_data['speaker_text'].apply(number_count)
speaker_data = speaker_data.drop(['speaker_text'], axis=1)

In [3]:
speaker_data

,file_name,last_update,speaker_number,context,section,count,sum,sent_count,sum_6,sum_num
0,1000000_T,2005-07-29 19:37:34+00:00,1,pres,1,1,67,6,18,0
1,1000000_T,2005-07-29 19:37:34+00:00,2,pres,1,1,373,16,119,8
2,1000000_T,2005-07-29 19:37:34+00:00,3,pres,1,1,1010,46,334,28
3,1000000_T,2005-07-29 19:37:34+00:00,4,pres,1,1,753,33,257,8
4,1000000_T,2005-07-29 19:37:34+00:00,5,pres,1,1,804,40,284,5
...,...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,74,qa,1,1,12,4,0,0
79,1000000_T,2005-07-29 19:37:34+00:00,75,qa,1,1,5,1,0,0
80,1000000_T,2005-07-29 19:37:34+00:00,76,qa,1,1,7,1,0,0
81,1000000_T,2005-07-29 19:37:34+00:00,77,qa,1,1,14,1,3,0


In [4]:
sql =  """
SELECT *
FROM se_features.word_counts
WHERE file_name = '1000000_T' 
ORDER BY section, context, speaker_number"""

orig_data = pd.read_sql(sql, engine)

In [5]:
orig_data

,file_name,last_update,speaker_name,speaker_number,context,section,count,sum,sent_count,sum_6,sum_num
0,1000000_T,2005-07-29 19:37:34+00:00,Operator,1,pres,1,1,67,6,18,0
1,1000000_T,2005-07-29 19:37:34+00:00,Steve Kyle,2,pres,1,1,373,16,119,8
2,1000000_T,2005-07-29 19:37:34+00:00,Carl Laurino,3,pres,1,1,1010,46,334,28
3,1000000_T,2005-07-29 19:37:34+00:00,Tim Kraus,4,pres,1,1,753,33,257,8
4,1000000_T,2005-07-29 19:37:34+00:00,Terry Growcock,5,pres,1,1,804,40,284,5
...,...,...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,Unidentified Company Representative,74,qa,1,1,12,4,0,0
79,1000000_T,2005-07-29 19:37:34+00:00,Operator,75,qa,1,1,5,1,0,0
80,1000000_T,2005-07-29 19:37:34+00:00,Steve Kyle,76,qa,1,1,7,1,0,0
81,1000000_T,2005-07-29 19:37:34+00:00,Operator,77,qa,1,1,14,1,3,0


In [6]:
# Please ignore the `False` in column `last_update`.
# It is `False` for the whole column because `last_update` in orig_data is in datetime format 
# While `last_update` in the new speaker_data is in str format
# However, they should be the same after uploading to the server with the dtype specification.

# orig_data has one more column `speaker_name`.
speaker_data == orig_data.drop(['speaker_name'], axis=1)

,file_name,last_update,speaker_number,context,section,count,sum,sent_count,sum_6,sum_num
0,True,False,True,True,True,True,True,True,True,True
1,True,False,True,True,True,True,True,True,True,True
2,True,False,True,True,True,True,True,True,True,True
3,True,False,True,True,True,True,True,True,True,True
4,True,False,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
78,True,False,True,True,True,True,True,True,True,True
79,True,False,True,True,True,True,True,True,True,True
80,True,False,True,True,True,True,True,True,True,True
81,True,False,True,True,True,True,True,True,True,True
